# <font color='red'>**Libraries**</font>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import preprocessing

# <font color='red'>**Data loading and preprocess**</font>
## Useful functions

In [ ]:
def load_data(general_path, split):
    ade_dat = np.load(general_path + split + '/'+ "adenomaEmbeddings.npy")
    ade_lab = np.load(general_path + split + '/'+ "adenomaLabels.npy")
    ade_vid = np.load(general_path + split + '/'+ "adenomaVideos.npy") 

    hyp_dat = np.load(general_path + split + '/'+ "hiperplasticEmbeddings.npy")
    hyp_lab = np.load(general_path + split + '/'+ "hiperplasticLabels.npy")
    hyp_vid = np.load(general_path + split + '/'+ "hiperplasticVideos.npy")
    
    print("==== "+ split + " data info ====")
    print("ade dim: {}, amount of labels: {}, videos: {}".format(ade_dat.shape, ade_lab.shape, ade_vid.shape))
    print("hyp dim: {}, amount of labels: {}, videos: {}".format(hyp_dat.shape, hyp_lab.shape, hyp_vid.shape))
    
    features = np.concatenate((ade_dat, hyp_dat), axis=0)
    labels = np.concatenate((ade_lab, hyp_lab), axis=0)
    videos = np.concatenate((ade_vid, hyp_vid), axis=0)
    
    df = pd.DataFrame({'features': list(features), 'label': labels, 'video': videos}, columns=['features', 'label', 'video'])
    
    return df

In [ ]:
def get_features(df):
    features = []
    for i in range(len(df)):
        tmp_features = df.loc[i]['features']
        features.append(tmp_features)

    features = np.array(features)
    
    return features

**Main**

In [ ]:
experiment = 'fineTunedModel'
general_path = "../../../unconditional/cycleGan-polyps/data/embeddings/adeVsHyp/embcVariation/"+ experiment + "/" 

In [ ]:
train_df = load_data(general_path, split='train')
train_df['info'] = train_df['label'] + '_' + train_df['video']
print("train_df info:")
print(train_df.groupby(['label']).count())

test_df = load_data(general_path, split='test')
test_df['info'] = test_df['label'] + '_' + test_df['video']
print("test_df info:")
print(test_df.groupby(['label']).count())

In [ ]:
test_df

# <font color='red'>**Loading and predicting test samples with the best model**</font>
## Model loading

In [ ]:
print("experiment ",experiment)
if experiment == 'embcBaseline':
    to_load = "../models_emb_classification/adeVsHyp/embcBaseline/fold1/test/KNN5.pkl"
else:
    to_load = "../models_emb_classification/adeVsHyp/embcVariation/fineTunedModel/test/RF10.pkl"
    
model = joblib.load(to_load)
print("model loaded!")

### Loading test samples

In [ ]:
general_path

In [ ]:
path = general_path + "test/"

ade_dat = np.load(path+"adenomaEmbeddings.npy")
ade_lab = np.load(path+"adenomaLabels.npy")
ade_vid = np.load(path+"adenomaVideos.npy") 

hyp_dat = np.load(path+"hiperplasticEmbeddings.npy")
hyp_lab = np.load(path+"hiperplasticLabels.npy")
hyp_vid = np.load(path+"hiperplasticVideos.npy") 

ser_dat = np.load(path+"serratedEmbeddings.npy")
ser_lab = np.load(path+"serratedLabels.npy")
ser_vid = np.load(path+"serratedVideos.npy") 

print("==== adenoma data info ====")
print("ade dim: {}, amount of labels: {}, videos: {}".format(ade_dat.shape, ade_lab.shape, ade_vid.shape))
print("==== hyperplastic data info ====")
print("hyp dim: {}, amount of labels: {}, videos: {}".format(hyp_dat.shape, hyp_lab.shape, hyp_vid.shape))
print("==== serrated data info ====")
print("ser dim: {}, amount of labels: {}, videos: {}".format(ser_dat.shape, ser_lab.shape, ser_vid.shape))

test_features = np.concatenate((ade_dat, hyp_dat), axis=0)
test_labels = np.concatenate((ade_lab, hyp_lab), axis=0)
test_videos = np.concatenate((ade_vid, hyp_vid), axis=0)

test_df = pd.DataFrame({'features': list(test_features), 'label': test_labels, 'video': test_videos},
                       columns=['features', 'label', 'video'])

ser_df = pd.DataFrame({'features': list(ser_dat), 'label': ser_lab, 'video': ser_vid},
                       columns=['features', 'label', 'video'])

In [ ]:
test_features = get_features(test_df)
print("test features shape: {}, min and max values: {} {}".format(test_features.shape, test_features.min(),
                                                                   test_features.max()))

ser_features = get_features(ser_df)
print("serrated features shape: {}, min and max values: {} {}".format(ser_features.shape, ser_features.min(),
                                                                      ser_features.max()))

### Testing

In [ ]:
print("test shape: ",test_features.shape)
print("ser shape: ",ser_features.shape)

In [ ]:
x_test, y_test = test_features, test_df['label'].values
x_test2, y_test2 = ser_features, ser_df['label'].values

In [ ]:
print("test labels info")
(unique, counts) = np.unique(y_test, return_counts=True)
print({x:y for x,y in zip(unique, counts)})
print("======================")
print("serrated labels info")
(unique, counts) = np.unique(y_test2, return_counts=True)
print({x:y for x,y in zip(unique, counts)})

### Predicting over adenoma and hyperplatic samples

In [ ]:
print("model prediction over test samples:")
y_pred = model.predict(x_test)

#print("total of preds: ", len(y_pred))
#(unique, counts) = np.unique(y_pred, return_counts=True)
#{x:y for x,y in zip(unique, counts)}

ade_pred_probs, hyp_pred_probs = [], []
probs = model.predict_proba(x_test)

for i in range(len(y_pred)):
    indx = y_pred[i]
    current_prob = probs[i][indx]
    if indx == 0:
        ade_pred_probs.append(current_prob)
    else:
        hyp_pred_probs.append(current_prob) 
        
ade_pred_probs = np.array(ade_pred_probs)
hyp_pred_probs = np.array(hyp_pred_probs)
        
print("probs sum: ", len(ade_pred_probs)+len(hyp_pred_probs))

#### <font color='red'>Saving predictions</font>

In [ ]:
print("real label: ", y_test.shape)
print("ade predictions: ", ade_pred_probs.shape)
print("hyp predictions: ", hyp_pred_probs.shape)
print("pred labels: ", y_pred.shape)

In [ ]:
ade_hyp_preds = np.concatenate((ade_pred_probs, hyp_pred_probs), axis=0)
ade_hyp_preds = ade_hyp_preds.reshape(len(ade_hyp_preds), 1)
y_pred = y_pred.reshape(len(y_pred), 1)
y_test = y_test.reshape(len(y_test), 1)

final_ade_hyp_preds = np.concatenate((y_test, ade_hyp_preds, y_pred), axis=1)
print(final_ade_hyp_preds.shape)

In [ ]:
final_ade_hyp_preds

In [ ]:
from numpy import savetxt

to_save = '../../embeddings/adeVsHyp/lossweighted2/predictions/ade_hyp_preds.csv'
savetxt(to_save, final_ade_hyp_preds, delimiter=',', fmt= '%s,%.3f,%i',header='real_label,probs,pred_label')
print("todo ok")

### Predicting over serrated samples

In [ ]:
print("model prediction over serrated samples:")
y_pred2 = model.predict(x_test2)

print("total of preds: ", len(y_pred2))
(unique, counts) = np.unique(y_pred2, return_counts=True)
{x:y for x,y in zip(unique, counts)}

probs2 = model.predict_proba(x_test2)
ser_pred_probs = np.max(probs2, axis=1)
print("total serrated preds:", len(ser_pred_probs))

In [ ]:
print(unique)
print(counts)

#### <font color='red'>Saving predictions</font>

In [ ]:
print("real label: ", y_test2.shape)
print("ade predictions: ", ser_pred_probs.shape)
print("pred labels: ", y_pred2.shape)

In [ ]:
y_test2 = y_test2.reshape(len(y_test2), 1)
serrated_preds2 = ser_pred_probs.reshape(len(ser_pred_probs), 1)
y_pred2 = y_pred2.reshape(len(y_pred2), 1)

serrated_preds = np.concatenate((y_test2, serrated_preds2, y_pred2), axis=1)
print(serrated_preds.shape)

In [ ]:
from numpy import savetxt

to_save = '../data/embeddings/adeVsHyp/lossweighted2/predictions/ser_preds.csv'
savetxt(to_save, serrated_preds, delimiter=',', fmt= '%s,%.3f,%i',header='real_label,probs,pred_label')
print("todo ok")

## Metrics

In [ ]:
print(len(y_test))
print(len(y_pred))

In [ ]:
pred2string = {0:"adenoma",
               1:"hiperplastic"}
y_pred_lab = []
for i in range(len(y_pred)):
    y_pred_lab.append(pred2string[y_pred[i][0]])

In [ ]:
target_names = ['adenoma', 'hyperplastic']
print(classification_report(y_test, y_pred_lab, target_names=target_names))

In [ ]:
#=========== only for AdeVsHyp ===========
target_names = ['adenoma', 'hiperplastic']
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_lab, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp = disp.plot(include_values=True, cmap=plt.cm.Blues, xticks_rotation='horizontal', values_format='.2f')

plt.grid(False)
plt.show()

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_lab, normalize=None)
cm

In [ ]:
#for auc
fpr = {}
tpr = {}
thresh ={}
n_class = 2
le = preprocessing.LabelEncoder()
y_test_enc = le.fit_transform(y_test)
for j in range(n_class):
    fpr[j], tpr[j], thresh[j] = roc_curve(y_test_enc, probs[:,j], pos_label=j)
ade_auc, hyp_auc = auc(fpr[0], tpr[0]), auc(fpr[1], tpr[1])
gen_auc = roc_auc_score(y_test_enc, np.argmax(probs, axis=1)) 
print("gen_auc: ", gen_auc)

# <font color='red'>**Probabilities**</font>

In [ ]:
probs = np.concatenate((ade_pred_probs, hyp_pred_probs, ser_pred_probs), axis=0)
probs.shape

In [ ]:
pred_label = np.concatenate((y_pred.reshape(len(y_pred)), y_pred2.reshape(len(y_pred2))), axis=0)
pred_label.shape

In [ ]:
test_final = pd.concat([test_df, ser_df], axis=0)
test_final['probs'] = probs
test_final['pred_label'] = pred_label
test_final.tail()

In [ ]:
test_final.groupby(['label']).count()

In [ ]:
test_final['pred_label'] = test_final['pred_label'].replace([0,1], ['adenoma', 'hiperplastic'])
test_final.tail()

In [ ]:
predictions = []
for i in range(len(test_final)):
    pred = test_final.iloc[i]['pred_label']
    prob = test_final.iloc[i]['probs']
    if pred == 'adenoma':
        predictions.append(prob)
    else:
        prob = 1 - prob
        prob = float(format(prob, '.2f'))
        predictions.append(prob)

In [ ]:
test_final['ade_prob'] = predictions
test_final.groupby('label').count()

In [ ]:
means = test_final.groupby(['label'])['ade_prob'].mean()
print(round(means,3))

In [ ]:
print(model)

In [ ]:
ade_df = test_final[test_final['label']=='adenoma']
ser_df = test_final[test_final['label']=='serrated']
hyp_df = test_final[test_final['label']=='hiperplastic']

In [ ]:
ade_mean = ade_df['ade_prob'].mean()
print("adenoma mean: ", round(ade_mean, 3))
hyp_mean = hyp_df['ade_prob'].mean()
print("hyperplastic mean: ", round(hyp_mean, 3))
ser_mean = ser_df['ade_prob'].mean()
print("serrated mean: ", round(ser_mean, 3))

## <font color='red'>**knowing all the polyps probabilities to being as adenoma**</font>

In [ ]:
means = test_final.groupby(['label'])['ade_prob'].mean()
print(round(means,3))

# <font color='red'>For image violin analysis</font>

In [ ]:
general_path

In [ ]:
#experiment = 'embcBaseline'
general_path = '../imgs_results/binary/embcVariation/test/fineTunedModel/onlinePreds.csv'
test_final = pd.read_csv(general_path, header=None)
test_final.columns = ['info', 'num_frame', 'ade_prob', 'hyp_prob']

label = []
predicted = []

for i in range(len(test_final)):
    info = test_final.iloc[i]['info']
    etiqueta = info.split('_')[0]
    label.append(etiqueta)
    ade_prob = test_final.iloc[i]['ade_prob']
    
    if ade_prob > 0.5:
        pred = 'adenoma'
    else:
        pred = 'hyperplastic'
        
    predicted.append(pred)
    
test_final['label'] = label
test_final['predicted'] = predicted

In [ ]:
test_final.tail()

In [ ]:
test_final.groupby('label').count()

In [ ]:
ade_df = test_final[test_final['label']=='adenoma']
hyp_df = test_final[test_final['label']=='hiperplastic']
ser_df = test_final[test_final['label']=='serrated']

In [ ]:
ade_mean = ade_df['ade_prob'].mean()
print("ade mean: ", round(ade_mean,3))

hyp_mean = hyp_df['ade_prob'].mean()
print("hyp as ade mean: ", round(hyp_mean,3))

ser_mean = ser_df['ade_prob'].mean()
print("ser as ade mean: ", round(ser_mean,3))

In [ ]:
means = test_final.groupby(['label'])['ade_prob'].mean()
print(round(means,3))

In [ ]:
ser_df[ser_df['predicted']=='hyperplastic'].count()

## Violin plot

In [ ]:
import matplotlib.transforms as transforms

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

#plt.title(string, fontsize=14)
my_pal = [(234/255, 104/255, 106/255), (33/255, 222/255, 132/255), (70/255, 163/255, 229/255)]

sns.set(style="whitegrid")
sns.violinplot(data=test_final, x='label', y='ade_prob', palette=my_pal, inner="quartile")
sns.boxenplot(x=test_final["label"], y=test_final["ade_prob"], color="gray", width=0.05)
plt.scatter(x=range(len(means)), y=means, c='w')
plt.title("Embedding odds from model with polyp classifier betas")
plt.ylabel("Adenoma probability", fontsize=14)
plt.xlabel("Polyp class", fontsize=14)
#plt.savefig('violinImg_betas.png')
plt.show()

## P value

In [ ]:
from scipy.stats import mannwhitneyu
from scipy import stats

In [ ]:
#teniendo en cuenta todas las predicciones pero con valores relativos a adenoma 
ade = test_final[test_final['label']=='adenoma']
ade_prob = ade['ade_prob'].values
hyp = test_final[test_final['label']=='hiperplastic']
hyp_prob = hyp['ade_prob'].values
ser = test_final[test_final['label']=='serrated']
ser_prob = ser['ade_prob'].values

In [ ]:
# Stacked histogram with multiple airlines
names = ['ade', 'hyp', 'ser']
plt.hist([ade_prob, hyp_prob, ser_prob], bins = int(180/15), stacked=True);

In [ ]:
# List of five airlines to plot
polyp_labels = ['adenoma', 'hiperplastic', 'serrated']

# Iterate through the five airlines
for polyp_type in polyp_labels:
    # Subset to the airline
    subset = test_final[test_final['label'] == polyp_type]
    
    # Draw the density plot
    sns.distplot(subset['ade_prob'], hist = False, kde = True,
                 kde_kws = {'linewidth': 3},
                 label = polyp_type)
    
# Plot formatting
plt.legend(prop={'size': 16}, title = 'Polyp labels')
plt.title('Density plot with polyp types')
plt.xlabel('Probability(%)')
plt.ylabel('Density');
plt.show()

### Can the data be used for a parametric test?
According to the above results...no

For remminder:
Differences between parametric vs non-parametric tests:
1. **Parametric test:** The data population follows a normal distribution (z-test, t-test, ANOVA)
2. **Non-parametric test:** non asumption about the data distribution (chi-square, Mann-Whitney U test)

**Mann-Whitney U test**
Useful for:
1. Compare two mean samples from the same population 
2. To say if two mean samples are the same or not---> affirm if two populations are different or say if there are differences between the group's means.
Data features for Mann-Whitney U test application:
1. The depend variable (adenoma probability) must be measured continiuosly (from 0 to 1 probs)
2. The independ variable must be two groups (adenoma vs hyperplastic for example)
3. Observational independence for each group
4. Both depend and independ variables are distributed anormaly (no regular distribution)

    if p<=alpha then H0 is rejected otherwise H0 is accepted. **Here H0:** the distribution for a sample X is the same for a sample Y
    
**Note:** **We are comparing median** as two polyp types have similar shape of distribution (see histogram and density function). If two groups do not have similar shape of distribution, you should compare mean ranks.

In [ ]:
print("amount of probabilities by label:\n")
print("adenoma: {}, hyper: {} and serrated: {}".format(ade_prob.shape, hyp_prob.shape, ser_prob.shape))

In [ ]:
print("some statistics over populations:\n")
print("adenoma:")
print("mean: {}, std: {}".format(round(ade_prob.mean(),3), round(ade_prob.std(),3)))
print("hyperplastic:")
print("mean: {}, std: {}".format(round(hyp_prob.mean(),3), round(hyp_prob.std(),3)))
print("serrated:")
print("mean: {}, std: {}".format(round(ser_prob.mean(),3), round(ser_prob.std(),3)))

In [ ]:
#example for upsampling data
def manWhitneyTest(population_1, population_2):
    from sklearn.utils import resample
    
    if len(population_1)>len(population_2):
        max_sample = population_1
        min_sample = population_2
    else:
        max_sample = population_2
        min_sample = population_1
        
    upsample = resample(min_sample, replace=True, n_samples=len(max_sample), random_state=42)
   
    max_sample = sorted(max_sample, key=float)
    upsample = sorted(upsample, key=float)
    
    statistic, p_value = mannwhitneyu(x=max_sample, y=upsample, alternative='two-sided')
    print("statistic: ", statistic)
    print("p_value: ", p_value)
    
    if p_value < 0.05:
        print("the distributions are DIFFERENT")
    else:
        print("the distributions are THE SAME")
    return np.array(max_sample), np.array(upsample) 

In [ ]:
print("====== P value test for ade Vs hyp: ======")
print("data sorted:")
max_sample, min_sample = manWhitneyTest(ade_prob, hyp_prob)
print("max_sample----> mean:{}, std{}, min_sample----> mean:{}, std{}".
      format(round(max_sample.mean(),3), round(max_sample.std(),3), round(min_sample.mean(),3), round(min_sample.std(),3)))

print("====== P value test for ade Vs ser: ======")
print("data sorted:")
max_sample, min_sample = manWhitneyTest(ade_prob, ser_prob)
print("max_sample----> mean:{}, std{}, min_sample----> mean:{}, std{}".
      format(round(max_sample.mean(),3), round(max_sample.std(),3), round(min_sample.mean(),3), round(min_sample.std(),3)))

print("====== P value test for hyp Vs ser: ======")
print("data sorted:")
max_sample, min_sample = manWhitneyTest(hyp_prob, ser_prob)
print("max_sample----> mean:{}, std{}, min_sample----> mean:{}, std{}".
      format(round(max_sample.mean(),3), round(max_sample.std(),3), round(min_sample.mean(),3), round(min_sample.std(),3)))

For future reading manwhitney : 
* https://www.statstutor.ac.uk/resources/uploaded/mannwhitney.pdf
* https://sphweb.bumc.bu.edu/otlt/mph-modules/bs/bs704_nonparametric/bs704_nonparametric4.html
Alternativa a manWhitney: Kruskal-Wallis One-Way analysis of variance and chi test


**Chi square**

In [ ]:
 from sklearn.utils import resample

In [ ]:
def resampling_data(population_1, population_2):
    if len(population_1) > len(population_2):
        max_sample = population_1
        min_sample = population_2
    else:
        max_sample = population_2
        min_sample = population_1
    
    upsample = resample(min_sample, replace=True, n_samples=len(max_sample), random_state=42)
    max_sample = np.array(sorted(max_sample, key=float))
    upsample = np.array(sorted(upsample, key=float))
    
    return max_sample, upsample

In [ ]:
def _chiSquare_test_(data_experimental, data_theorical, alpha=0.05):
    """Function that execute the chi Square Test. In this case the theorical data is required to test the null hypothesis of 'experimental data follow the theorical data frequencies or distribution' and finally returns a boolean for the null hypothesis with the statistical value of the test. This methods is based in scipy chisquare method but its applied by hand.
    Args:
        data_experimental (Array): An 1D array of data containing the values to be tested.
        data_teorical (Array): An 1D array of data containing the expected values to be compared.
        alpha (Float): A decimal value meaning the significance level, default is 0.05 for 5%.
    """
    terms = (data_experimental - data_theorical)**2 / (data_theorical + 1e-3)
    statistic = np.sum(terms)
    p_value = stats.chi2.sf(statistic, data_theorical.shape[0] - 1)
    print("statistic: ", statistic)
    print("p_value: ", p_value)
    if p_value < alpha:
        return False, p_value, statistic
    else: 
        return True, p_value, statistic

In [ ]:
print("for adenoma Vs hyperplastic:")
max_sample, upsample = resampling_data(ade_prob, hyp_prob)
print(_chiSquare_test_(max_sample, upsample))
print("for adenoma Vs serrated:")
max_sample, upsample = resampling_data(ade_prob, ser_prob)
print(_chiSquare_test_(max_sample, upsample))
print("for hyperplastic Vs serrated:")
max_sample, upsample = resampling_data(hyp_prob, ser_prob)
print(_chiSquare_test_(max_sample, upsample))